In [1]:
#This version of the evolutionary algorithm seeks to implement
#local search and investigates the variance for hyperparameter tuning

In [1]:
import numpy as np
import time
from statistics import mean

In [2]:
def selection(pop, scores, k=3):
    selection_ix = np.random.randint(len(pop))
    for ix in np.random.randint(0, len(pop), k-1):
        if scores[ix] > scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if np.random.rand() < r_cross:
        pt = np.random.randint(1, len(p1)-1)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if np.random.rand() < r_mut:
            bitstring[i] = np.random.randint(0, 4)#1 - bitstring[i]
    return bitstring

In [3]:
#mutation([1] * 10 + [0] * 10, 1/4)

In [4]:
def update_consumergoods(state, toaster_percent):
    match state:
        case 7:
            toaster_percent -= 0.15 #WAR ECO     
        case 30:
            toaster_percent -= 0.05 #construction company
        case 34:
            toaster_percent -= 0.03 #stability bonus
    return round(toaster_percent, 2)

def update_civs(state, civs):
    match state:
        case 8:
            civs += 2
        case 20:
            civs += 2
        case 22:
            civs += 2
        case 25:
            civs += 2
        case 34:
            civs += 13 #annexation of baltics        
    return civs

def update_mils(state, mils):
    match state:
        case 34:
            mils += 8    
    return mils


def update_constructionspeed(state, civ_output_modifier, scenario):
    match scenario:
        case 'one year':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 12: #same as half year because we start in 36
                    civ_output_modifier += 0.1 
                case 31:
                    civ_output_modifier += 0.1
                case 52:
                    civ_output_modifier += 0.1
        case 'half year':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 12: 
                    civ_output_modifier += 0.1 
                case 33:
                    civ_output_modifier += 0.1
                case 54:
                    civ_output_modifier += 0.1
        case 'current':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 16: 
                    civ_output_modifier += 0.1 
                case 36:
                    civ_output_modifier += 0.1
                case 57:
                    civ_output_modifier += 0.1
            
    return round(civ_output_modifier, 2)

def update_miloutput(state, mil_output_modifier, scenario):
    match state: #These are the bonuses related to stability
        case 0:
            mil_output_modifier -= 0.2180
        case 2: 
            mil_output_modifier += 0.05
        case 4: 
            mil_output_modifier += 0.05
        case 8:
            mil_output_modifier += 0.10
        case 9: #At this point we are above 50% stability
            mil_output_modifier += 0.04
        case 10: #popular figurehead
            mil_output_modifier += 0.06 
        case 24: #kill trotsky + new NKVD head
            mil_output_modifier += 0.04
    
    match scenario:
        case 'one year':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 14: 
                    mil_output_modifier += 0.15
                case 31:
                    mil_output_modifier += 0.15
                case 52:
                    mil_output_modifier += 0.15
        case 'half year':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 14: 
                    mil_output_modifier += 0.15
                case 33:
                    mil_output_modifier += 0.15
                case 54:
                    mil_output_modifier += 0.15
        case 'current':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 16: 
                    mil_output_modifier += 0.15
                case 36:
                    mil_output_modifier += 0.15
                case 57:
                    mil_output_modifier += 0.15

    return round(mil_output_modifier, 3)

def update_prodefficiency(state, production_efficiency_cap, scenario):
    match state: #These are related to the 'defense industry' buff in the focus tree
        case 30:
            production_efficiency_cap += 0.05
        case 40:
            production_efficiency_cap += 0.05
    
    match scenario:
        case 'one year':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 13:
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 31:
                    production_efficiency_cap += 0.1
                case 52:
                    production_efficiency_cap += 0.1
        case 'half year':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 13: 
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 33:
                    production_efficiency_cap += 0.1
                case 54:
                    production_efficiency_cap += 0.1
        case 'current':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 15: 
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 36:
                    production_efficiency_cap += 0.1
                case 57:
                    production_efficiency_cap += 0.1

    return round(production_efficiency_cap, 2)

def update_civcost(state, civ_construction_modifier):
    match state:
        case 0:
            civ_construction_modifier += 0.05 #five year plan
        case 7:
            civ_construction_modifier += 0.3
        case 12: 
            civ_construction_modifier += 0.1 #captain of industry
        case 21:
            civ_construction_modifier += 0.05 #construction company
        case 30:
            civ_construction_modifier += 0.05 #construction company upgrade
    return round(civ_construction_modifier, 2)

def update_milcost(state, mil_construction_modifier):
    match state:
        case 7:
            mil_construction_modifier += 0.5
        case 21:
            mil_construction_modifier += 0.05 #construction company
    return round(mil_construction_modifier, 2)

def update_civ_to_milcost(state, civ_to_mil_construction_modifier):
    match state:
        case 7:
            civ_to_mil_construction_modifier += 0.5
    return round(civ_to_mil_construction_modifier, 2)

def update_mil_to_civcost(state, mil_to_civ_construction_modifier):
    match state:
        case 7:
            mil_to_civ_construction_modifier += 0.5
    return round(mil_to_civ_construction_modifier, 2)

def prod_eff_calculations(state, prod_eff, prod_eff_cap, factory_state_array, new_mils):
    factory_count_list = []
    efficiency_list = []
    sum_of_efficiency = 0
    prod_eff = 0
    factory_state_array[state][0] = new_mils
    factory_state_array[state][1] = 0.1
    
    if state == 0:
        factory_state_array[state][1] = 0.5
    
    for i in range(state):
        factory_count_list.append(factory_state_array[i][0])
        efficiency_list.append(factory_state_array[i][1])   
        growth = 0.001 * (prod_eff_cap ** 2 / factory_state_array[i][1]) * 17.5
        factory_state_array[i][1] += growth
        factory_state_array[i][1] = min(factory_state_array[i][1], 1)
        
    num_factories = sum(factory_count_list)
    
    for j in range(len(factory_count_list)):
        sum_of_efficiency += factory_count_list[j] * efficiency_list[j]
    
    try:
        return min(sum_of_efficiency / num_factories, 1)
    except:
        return 0.5 #This should only happen at the start, then eff = 0.5

In [5]:
#Five max_targets: 'rush', 'scale', 'scale_total', 'scale_civs' and 'mils'

def production(actions, max_target, scenario):
    
    st = time.time()
    mils = 32
    mils_0 = 32 #keep track of the number of starting factories
    new_mils = 32 #initialize production efficiency
    civs = 45 + 8 #(a baseline number of factories from trade)
    civs_0 = 45 #idem
    total = mils + civs
    total_0 = 32 + 45 #idem
    consumergoods_percentage = 0.40
    consumergoods = np.floor(total * consumergoods_percentage)
    available_civs = np.maximum(int(civs - consumergoods), 0)
    
    #Civ output modifiers are additive (-30% + 15% = -15%), multiplied by state infrastructure
    civ_output = 5 
    civ_output_modifier = 1 #Construction speed
    civ_construction_modifier = 0.7
    mil_construction_modifier = 0.7
    civ_to_mil_construction_modifier = 0.7
    mil_to_civ_construction_modifier = 0.7
    
    mil_output = 4.5
    mil_output_modifier = 1
    prod_eff = 0.5
    production_efficiency_cap = 0.55
    
    state = 0
    state_size_days = 35 #how many days per state
    ic_produced = 0
    ic_total = 0    
    civ_purchase_pool = 0 
    mil_purchase_pool = 0
    civ_to_mil_pool = 0
    mil_to_civ_pool = 0
    infrastructure_modifier = 1.6
    
    factory_state_array = np.empty((57), dtype=object)
    #prod_list = []

    for i in range(len(factory_state_array)):
        factory_state_array[i] = np.zeros(2)
    
    for i in range(len(actions)):
        
        civs = (update_civs(i, civs))
        mils = (update_mils(i, mils))
        consumergoods_percentage = (update_consumergoods(i, consumergoods_percentage))
        available_civs = np.maximum(int(civs - consumergoods), 0)
        civ_output_modifier = (update_constructionspeed(i, civ_output_modifier, scenario))
        mil_output_modifier = (update_miloutput(i, mil_output_modifier, scenario))
        civ_construction_modifier = (update_civcost(i, civ_construction_modifier))
        mil_construction_modifier = (update_milcost(i, mil_construction_modifier))
        civ_to_mil_construction_modifier = (update_civ_to_milcost(i, civ_to_mil_construction_modifier))
        mil_to_civ_construction_modifier = (update_mil_to_civcost(i, mil_to_civ_construction_modifier))
        production_efficiency_cap = (update_prodefficiency(i, production_efficiency_cap, scenario))
        
        ic_produced = mil_output * mils * state_size_days * mil_output_modifier * prod_eff
        prod_eff = prod_eff_calculations(i, prod_eff, production_efficiency_cap, factory_state_array, new_mils)
        #prod_list.append(civs)

        if actions[i] == 0: #build mils     
            mil_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_construction_modifier)
            existing_mils = mils #Used exclusively for production efficiency calculations
            new_mils = np.floor_divide(mil_purchase_pool, 7200) #for production efficiency
            mils += np.floor_divide(mil_purchase_pool, 7200) 
            total += np.floor_divide(mil_purchase_pool, 7200) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            mil_purchase_pool = np.mod(mil_purchase_pool, 7200)
            available_civs = np.maximum(int(civs - consumergoods), 0)
                     
            state += 1
            ic_total += ic_produced
            
        if actions[i] == 1: #build civs         
            civ_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_construction_modifier)

            new_mils = 0 #Account for annexations here
            civs += np.floor_divide(civ_purchase_pool, 10800) 
            total += np.floor_divide(civ_purchase_pool, 10800) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            civ_purchase_pool = np.mod(civ_purchase_pool, 10800)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            
            state += 1
            ic_total += ic_produced

        if actions[i] == 2: #Convert civs to mils
            civ_to_mil_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_to_mil_construction_modifier)
            new_mils = np.floor_divide(civ_to_mil_pool, 4000)
            civs -= np.floor_divide(civ_to_mil_pool, 4000)
            mils += np.floor_divide(civ_to_mil_pool, 4000)
            civ_to_mil_pool = np.mod(civ_to_mil_pool, 4000)
            consumergoods = np.ceil(total * consumergoods_percentage)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
        
        if actions[i] == 3: #Convert mils to civs
            mil_to_civ_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_to_civ_construction_modifier)
            if mils > 0:
                new_mils = 0
                civs += np.floor_divide(mil_to_civ_pool, 9000)
                mils -= np.floor_divide(mil_to_civ_pool, 9000)
                
                if mils < 0:
                    too_many_removed = abs(mils)
                    civs -= too_many_removed
                    mils = 0
                    mil_to_civ_pool = 0
                
                else:
                    mil_to_civ_pool = np.mod(mil_to_civ_pool, 9000)
                consumergoods = np.ceil(total * consumergoods_percentage)
                available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
            
    et = time.time()
    elapsed_time = et - st
    #print("Time taken: " + str(elapsed_time) + " seconds")
            
    match max_target:
        case 'rush':
            return ((ic_total, mils, civs))
        case 'scale':
            scaled_score = ic_total * np.log2((mils/mils_0))
            return ((scaled_score, mils, civs, ic_total))
        case 'scale_total':
            scaled_score = ic_total * np.log2((mils/mils_0)) * (total/total_0)
            if scaled_score == None:
                scaled_score = 0
            return ((scaled_score, mils, civs, ic_total))
        case 'scale_civs':
            scaled_score = ic_total * np.log2((mils/mils_0)) * np.log2((civs/civs_0))
            if scaled_score == None:
                scaled_score = 0
            return ((scaled_score, mils, civs, ic_total))
        case 'mils':
            return ((mils, civs, ic_total))

In [6]:
def genetic_algorithm(n_bits, n_pop, n_iter, r_cross, max_target, scenario, mutation_rate, elitism, migration):
    
    st = time.time()
    pop = [np.random.randint(0, 4, n_bits).tolist() for _ in range(n_pop)]
    best, best_eval = 0, production(pop[0], max_target, scenario)
    gen_since_last_improvement = 0
    
    for gen in range(n_iter):
        #print('generation ' + str(gen) + ' population: ' + str(pop))
        if gen_since_last_improvement < 10:
            gen_since_last_improvement += 1
            scores = [production(c, max_target, scenario) for c in pop]
            for i in range(n_pop):
                if scores[i] > best_eval:
                    best, best_eval = pop[i], scores[i]
                    gen_since_last_improvement = 0
                    #print("Generation " + str(gen) + ": new best solution found!")
                    #print(best, best_eval)
                
        
            selected = [selection(pop, scores) for _ in range(n_pop)]
            children = []
            for i in range(0, n_pop, 2):
                p1, p2 = selected[i], selected[i+1]
                c = crossover(p1, p2, r_cross)
                for child in c:
                    mutated = mutation(child, mutation_rate)
                    children.append(mutated)
                    
            #Add random new samples
            if gen % 10 == 0 and gen > 0 and migration:
                del children[-10:]
                migration = [np.random.randint(0, 4, n_bits).tolist() for _ in range(10)]
                for i in migration:
                    children.append(i)
            
            if gen > 1 and elitism:
                del children[-1] #Delete the last element and replace with best solution
                children.append(best)

            pop = children
        else:
            break

    et = time.time()
    elapsed_time = et - st
    print("Time taken: " + str(elapsed_time) + " seconds")
    
    return [best, best_eval]

In [7]:
def local_search(strategy, strat_result, max_target, scenario):
    possible_actions = [0, 1, 2, 3]
    best_strat = strategy
    best_result = strat_result
    print('Starting point: ')
    print(strategy, strat_result)
    for i in range(len(strategy)):
        for j in possible_actions:
            new_strat = strategy.copy()
            if new_strat[i] != j:
                new_strat[i] = j
                new_strat_prod = production(new_strat, max_target, scenario)
                if new_strat_prod[0] > best_result[0]:
                    print('Improved strategy found with local search: ')
                    print(new_strat, new_strat_prod[0], strategy, best_result[0])
                    best_strat = new_strat
                    best_result = new_strat_prod
    
    
    #print(best_strat, best_result)
    #local_search(best_strat, best_result)
    
    return (best_strat, best_result)



def attempts_local_search(strat, strat_result, max_target, scenario):
    attempts = 0

    st = time.time()

    while attempts < 10000:
    
        iteration = local_search(strat, strat_result, max_target, scenario)
    
        if strat_result[0] - iteration[1][0] == 0:
            break
    
        strat = iteration[0]
        strat_result = iteration[1]
    
        attempts += 1
    
    et = time.time()
    elapsed_time = et - st
    print("Time taken: " + str(elapsed_time) + " seconds")
    return (strat, strat_result)

In [8]:
rush_strat = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
              0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2]
scale_total_strat = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                     0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
scale_civs_strat = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                    0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]
human_rush_strat = [1] * 25 + [0] * 32
human_scale_strat = [1] * 35 + [0] * 22

rush_result = production(rush_strat, 'rush', 'half year')
scale_total_result = production(scale_total_strat, 'scale_total', 'half year')
scale_civs_result = production(scale_civs_strat, 'scale_civs', 'half year')
human_rush_result = production(human_rush_strat, 'rush', 'half year')
human_scale_result = production(human_scale_strat, 'scale_total', 'half year')

In [9]:
attempts_local_search(rush_strat, rush_result, 'rush', 'half year'), rush_result

Starting point: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] (796905.0566316111, 161.0, 38.0)
Time taken: 0.7029728889465332 seconds


(([0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2],
  (796905.0566316111, 161.0, 38.0)),
 (796905.0566316111, 161.0, 38.0))

In [10]:
attempts_local_search(scale_total_strat, scale_total_result, 'scale_total', 'half year'), scale_total_result

Starting point: 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (6158048.018822445, 191.0, 118.0, 657035.8749602238)
Time taken: 0.6911516189575195 seconds


(([1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  (6158048.018822445, 191.0, 118.0, 657035.8749602238)),
 (6158048.018822445, 191.0, 118.0, 657035.8749602238))

In [38]:
attempts_local_search(scale_civs_strat, scale_civs_result, 'scale_civs', 'half year'), scale_civs_result

Starting point: 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0] (2543185.812488124, 167.0, 157.0, 591814.9840359812)
Time taken: 0.6632475852966309 seconds


(([1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  (2543185.812488124, 167.0, 157.0, 591814.9840359812)),
 (2543185.812488124, 167.0, 157.0, 591814.9840359812))

In [36]:
#genetic_algorithm(57,100, 600, 0.8, 'scale_total', 'half year', 1 / 57, True, False)

In [11]:
test_parameters = [(6151354.4969999995, 500, 0.8, 0.017543859649122806, True, False), 
                   (6151036.403, 500, 1, 0.017543859649122806, True, True),
                   (6148971.013, 500, 1, 0.01, False, True), (4520889.74, 100, 0.8, 0.1, False, False), 
                   (4601687.935, 100, 0.9, 0.1, False, True), (4620992.946, 100, 1, 0.1, False, False)]

In [12]:
def hyperparameter_variance(parameter_list):
    
    nested_list_results = []
    for i in range(len(test_parameters)):
        list_results = []
        for j in range(10):
            result = genetic_algorithm(57, parameter_list[i][1], 600, parameter_list[i][2], 
                                       'scale_total', 'half year', parameter_list[i][3],
                                       parameter_list[i][4], parameter_list[i][5])
            list_results.append((result[1][0], parameter_list[i][1], parameter_list[i][2], 
                                 parameter_list[i][3], parameter_list[i][4], parameter_list[i][5]))
        nested_list_results.append(list_results)
    
    return nested_list_results

In [54]:
#hyperparameter_variance(test_parameters)

Time taken: 156.87007308006287 seconds
Time taken: 145.88894653320312 seconds
Time taken: 138.63510966300964 seconds
Time taken: 136.77309012413025 seconds
Time taken: 116.8082160949707 seconds
Time taken: 148.95320343971252 seconds
Time taken: 153.7614710330963 seconds
Time taken: 117.72161197662354 seconds
Time taken: 144.6092586517334 seconds
Time taken: 146.40792393684387 seconds
Time taken: 139.42922353744507 seconds
Time taken: 129.18670439720154 seconds
Time taken: 161.53405714035034 seconds
Time taken: 168.41747975349426 seconds
Time taken: 133.05506372451782 seconds
Time taken: 121.73133063316345 seconds
Time taken: 154.54842567443848 seconds
Time taken: 153.1252942085266 seconds
Time taken: 125.33471775054932 seconds
Time taken: 110.5115613937378 seconds
Time taken: 123.46251082420349 seconds
Time taken: 127.83342146873474 seconds
Time taken: 113.08142066001892 seconds
Time taken: 165.3051028251648 seconds
Time taken: 124.28811073303223 seconds
Time taken: 117.25217938423157 

[[(6158048.018822445, 500, 0.8, 0.017543859649122806, True, False),
  (6158048.018822445, 500, 0.8, 0.017543859649122806, True, False),
  (6158048.018822445, 500, 0.8, 0.017543859649122806, True, False),
  (6144782.047397442, 500, 0.8, 0.017543859649122806, True, False),
  (6136243.464103532, 500, 0.8, 0.017543859649122806, True, False),
  (6144735.874566358, 500, 0.8, 0.017543859649122806, True, False),
  (6144782.047397442, 500, 0.8, 0.017543859649122806, True, False),
  (6144735.874566358, 500, 0.8, 0.017543859649122806, True, False),
  (6115921.715058356, 500, 0.8, 0.017543859649122806, True, False),
  (6144782.047397442, 500, 0.8, 0.017543859649122806, True, False)],
 [(6145751.013193969, 500, 1, 0.017543859649122806, True, True),
  (6124237.107838373, 500, 1, 0.017543859649122806, True, True),
  (6158048.018822445, 500, 1, 0.017543859649122806, True, True),
  (6142995.545171711, 500, 1, 0.017543859649122806, True, True),
  (6158048.018822445, 500, 1, 0.017543859649122806, True, T

In [13]:
result_data = [[(6158048.018822445, 500, 0.8, 0.017543859649122806, True, False),
  (6158048.018822445, 500, 0.8, 0.017543859649122806, True, False),
  (6158048.018822445, 500, 0.8, 0.017543859649122806, True, False),
  (6144782.047397442, 500, 0.8, 0.017543859649122806, True, False),
  (6136243.464103532, 500, 0.8, 0.017543859649122806, True, False),
  (6144735.874566358, 500, 0.8, 0.017543859649122806, True, False),
  (6144782.047397442, 500, 0.8, 0.017543859649122806, True, False),
  (6144735.874566358, 500, 0.8, 0.017543859649122806, True, False),
  (6115921.715058356, 500, 0.8, 0.017543859649122806, True, False),
  (6144782.047397442, 500, 0.8, 0.017543859649122806, True, False)],
 [(6145751.013193969, 500, 1, 0.017543859649122806, True, True),
  (6124237.107838373, 500, 1, 0.017543859649122806, True, True),
  (6158048.018822445, 500, 1, 0.017543859649122806, True, True),
  (6142995.545171711, 500, 1, 0.017543859649122806, True, True),
  (6158048.018822445, 500, 1, 0.017543859649122806, True, True),
  (6144735.874566358, 500, 1, 0.017543859649122806, True, True),
  (6158048.018822445, 500, 1, 0.017543859649122806, True, True),
  (6158048.018822445, 500, 1, 0.017543859649122806, True, True),
  (6144782.047397442, 500, 1, 0.017543859649122806, True, True),
  (6112267.2094439585, 500, 1, 0.017543859649122806, True, True)],
 [(6110752.405182425, 500, 1, 0.01, False, True),
  (6144735.874566358, 500, 1, 0.01, False, True),
  (6158048.018822445, 500, 1, 0.01, False, True),
  (6158048.018822445, 500, 1, 0.01, False, True),
  (6158048.018822445, 500, 1, 0.01, False, True),
  (6144735.874566358, 500, 1, 0.01, False, True),
  (6136243.464103532, 500, 1, 0.01, False, True),
  (6133326.034509568, 500, 1, 0.01, False, True),
  (6158048.018822445, 500, 1, 0.01, False, True),
  (6158048.018822445, 500, 1, 0.01, False, True)],
 [(4623797.131331957, 100, 0.8, 0.1, False, False),
  (4659008.0239548925, 100, 0.8, 0.1, False, False),
  (4784848.590788559, 100, 0.8, 0.1, False, False),
  (4514848.699419382, 100, 0.8, 0.1, False, False),
  (4856977.646090645, 100, 0.8, 0.1, False, False),
  (4832005.750043164, 100, 0.8, 0.1, False, False),
  (4487784.033780217, 100, 0.8, 0.1, False, False),
  (4643421.606592335, 100, 0.8, 0.1, False, False),
  (4394053.211933742, 100, 0.8, 0.1, False, False),
  (4571552.082995911, 100, 0.8, 0.1, False, False)],
 [(4766368.540499983, 100, 0.9, 0.1, False, True),
  (4747936.079216426, 100, 0.9, 0.1, False, True),
  (4811017.964873091, 100, 0.9, 0.1, False, True),
  (5049983.46648918, 100, 0.9, 0.1, False, True),
  (4575894.271602228, 100, 0.9, 0.1, False, True),
  (4431839.045121, 100, 0.9, 0.1, False, True),
  (4777257.393734956, 100, 0.9, 0.1, False, True),
  (4742448.150871407, 100, 0.9, 0.1, False, True),
  (4509589.619024683, 100, 0.9, 0.1, False, True),
  (4616096.796934165, 100, 0.9, 0.1, False, True)],
 [(4841097.396052668, 100, 1, 0.1, False, False),
  (4618640.071675427, 100, 1, 0.1, False, False),
  (4591795.654706923, 100, 1, 0.1, False, False),
  (4928381.7478472, 100, 1, 0.1, False, False),
  (4949011.612202699, 100, 1, 0.1, False, False),
  (4854709.4668746255, 100, 1, 0.1, False, False),
  (4755161.60693747, 100, 1, 0.1, False, False),
  (4766225.368303271, 100, 1, 0.1, False, False),
  (4522785.115359869, 100, 1, 0.1, False, False),
  (4808457.79032576, 100, 1, 0.1, False, False)]]